> # **TensorFlow  2.1.0** - ***04*** - *Transferência de Aprendizado e Fine Tuning*



In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [30]:
# Download do dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-02-25 15:21:56--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  38.9MB/s    in 1.7s    

2020-02-25 15:21:59 (38.9 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
# Descompactando e configurando paths do dataset
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

dataset_path = './cats_and_dogs_filtered'
training_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

In [0]:
# Carregando modeloo pré-treinando (MobileNetV2)
img_shape = (128, 128, 3) # MobileNetV2 suporta: (96, 96), (128, 128), (160, 160), (192, 129), (224, 224)
# include_top = False -> cabeçalho(saida) personalizado | include_top = True -> usar toda estrutura 
# (deve ter compatibilidade na saida, se no MobileNetV2 tiver 1000 saidas, nossa rede tambem deve ter 1000 saidas)
base_model = tf.keras.applications.MobileNetV2(input_shape= img_shape, include_top= False, weights= 'imagenet')

In [33]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
# Congelando modelo base
base_model.trainable = False # Não vai alterar os pesos

In [35]:
# Definindo cabeçalho personalizado da rede neural (camada de saida)
print(base_model.output)

Tensor("out_relu_1/Identity:0", shape=(None, 4, 4, 1280), dtype=float32)


In [36]:
# Reduz a dimensionalidade dos dados
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
print(global_average_layer)

Tensor("global_average_pooling2d_3/Identity:0", shape=(None, 1280), dtype=float32)


In [37]:
# camada de saida
prediction_layer = tf.keras.layers.Dense(units= 1, activation= 'sigmoid')(global_average_layer)

# Definindo modelo
model = tf.keras.models.Model(inputs= base_model.input, outputs= prediction_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
# Compilando modelo
model.compile(optimizer= tf.keras.optimizers.RMSprop(lr= 0.0001), 
              loss= 'binary_crossentropy', metrics= ['accuracy'])

In [39]:
# Data Generetos
data_gen_training = ImageDataGenerator(rescale= 1/255.)
data_gen_validation = ImageDataGenerator(rescale= 1/255.)

training_generator = data_gen_training.flow_from_directory(training_dir, target_size= (128, 128),
                                                           batch_size= 128, class_mode= 'binary')

validation_generator = data_gen_validation.flow_from_directory(validation_dir, target_size= (128, 128),
                                                           batch_size= 128, class_mode= 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [41]:
# Treinamento do modelo
model.fit_generator(training_generator, epochs= 10, validation_data= validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/10
16/16 [==============================] - 8s 511ms/step - loss: 0.4873 - accuracy: 0.7875 - val_loss: 0.4114 - val_accuracy: 0.8380
Epoch 2/10
16/16 [==============================] - 8s 517ms/step - loss: 0.4550 - accuracy: 0.8090 - val_loss: 0.3823 - val_accuracy: 0.8470
Epoch 3/10
16/16 [==============================] - 8s 516ms/step - loss: 0.4261 - accuracy: 0.8310 - val_loss: 0.3612 - val_accuracy: 0.8520
Epoch 4/10
16/16 [==============================] - 8s 504ms/step - loss: 0.4009 - accuracy: 0.8430 - val_loss: 0.3411 - val_accuracy: 0.8590
Epoch 5/10
16/16 [==============================] - 8s 497ms/step - loss: 0.3780 - accuracy: 0.8570 - val_loss: 0.3163 - val_accuracy: 0.8760
Epoch 6/10
16/16 [==============================] - 8s 500ms/step - loss: 0.3577 - accuracy: 0.8650 - val_loss: 0.3091 - val_accuracy: 0.8740
Epoch 7/10
16/16 [==============================] - 8s 51

In [44]:
# Avaliação do modelo
valid_loss, valid_accuracy = model.evaluate_generator(validation_generator)
print(f'valididation_accuracy - {valid_accuracy}')
print(f'valididation_loss - {valid_loss}')

  ...
    to  
  ['...']
valididation_accuracy - 0.8980000019073486
valididation_loss - 0.2577111180871725


In [45]:
# Fine Tuning
base_model.trainable = True
print(len(base_model.layers))

155


In [0]:
fine_tunning_at = 100
for layer in base_model.layers[:fine_tunning_at]:
  layer.trainable = False

In [0]:
# Compilando o modelo para Fine Tuning
model.compile(optimizer= tf.keras.optimizers.RMSprop(lr= 0.0001), loss= 'binary_crossentropy',
              metrics= ['accuracy'])

In [51]:
# Aplicando o Fine Tuning
model.fit_generator(training_generator, epochs= 10, validation_data= validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/10
16/16 [==============================] - 8s 522ms/step - loss: 1.0627e-04 - accuracy: 1.0000 - val_loss: 0.1552 - val_accuracy: 0.9620
Epoch 2/10
16/16 [==============================] - 8s 524ms/step - loss: 4.3455e-05 - accuracy: 1.0000 - val_loss: 0.1661 - val_accuracy: 0.9620
Epoch 3/10
16/16 [==============================] - 8s 506ms/step - loss: 1.7670e-05 - accuracy: 1.0000 - val_loss: 0.1795 - val_accuracy: 0.9610
Epoch 4/10
16/16 [==============================] - 8s 502ms/step - loss: 7.3363e-06 - accuracy: 1.0000 - val_loss: 0.1993 - val_accuracy: 0.9620
Epoch 5/10
16/16 [==============================] - 8s 519ms/step - loss: 3.2622e-06 - accuracy: 1.0000 - val_loss: 0.2025 - val_accuracy: 0.9610
Epoch 6/10
16/16 [==============================] - 8s 503ms/step - loss: 1.5780e-06 - accuracy: 1.0000 - val_loss: 0.2183 - val_accuracy: 0.9610
Epoch 7/10
16/16 [===============

In [52]:
# Avaliação do modelo com Fine Tuning
valid_loss, valid_accuracy = model.evaluate_generator(validation_generator)
print(f'valididation_accuracy - {valid_accuracy}')
print(f'valididation_loss - {valid_loss}')

  ...
    to  
  ['...']
valididation_accuracy - 0.9610000252723694
valididation_loss - 0.250895619907169
